# Define the problem

You have five years of google stock price, your job is to predict the next stock price

# Obtain Data

In [0]:
!wget https://www.dropbox.com/s/m6xpec59d3o757t/GoogleStockPrice.zip -q

In [0]:
!unzip GoogleStockPrice.zip

In [0]:
ls -l

# Analyze the data

In [0]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
# Importing the training set
train_pd = pd.read_csv('Google_Stock_Price_Train.csv')


In [0]:
train_pd.info()

In [0]:
train_pd.head()

In [0]:
train_pd.tail()

In [0]:
# Importing the test set
test_pd = pd.read_csv('Google_Stock_Price_Test.csv')


In [0]:
test_pd.head()

In [0]:
test_pd.tail()

In [0]:
train_pd['Open'].plot(color='k')

In [0]:
test_pd['Open'].plot(color='k')

# Feature Engineering

In [0]:
training_set = train_pd.iloc[:, 1:2].values


In [0]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))


In [0]:
print(sc.data_max_,sc.data_min_,sc.data_range_)
# this will be an error, to show max and min are not set

In [0]:
sc.fit(training_set)


In [0]:
print(sc.data_max_,sc.data_min_,sc.data_range_)

In [0]:
training_set_scaled = sc.transform(training_set)


In [0]:
temp_sc = MinMaxScaler(feature_range = (0, 1))
temp_sc.fit(training_set_scaled)
print(temp_sc.data_max_,temp_sc.data_min_,temp_sc.data_range_)

In [0]:
training_set_scaled.shape

In [0]:
m = training_set_scaled.shape[0]

In [0]:
m

In [0]:
# Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []
for i in range(60, m):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)


In [0]:
X_train[0]

In [0]:
y_train[0]

In [0]:
X_train.shape

In [0]:
# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


In [0]:
X_train.shape

In [0]:
X_train[0]

# Build the Model

In [0]:
# Part 2 - Building the RNN / LSTM

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout


In [0]:
# Initialising the Nueral Net
regressor = Sequential()

In [0]:
# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 64, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
regressor.add(Dropout(0.2))


In [0]:
# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 64, return_sequences = True))
regressor.add(Dropout(0.2))

In [0]:
# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 64, return_sequences = True))
regressor.add(Dropout(0.2))

In [0]:
# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 64))
regressor.add(Dropout(0.2))

In [0]:
# Adding the output layer
regressor.add(Dense(units = 1)) # add Relu to avoid confusion.

In [0]:
# Compiling the RNN
# learning rate is defined in the default paramater for "Adam"
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error') # RMSprop could also be used for RNN
# https://keras.io/optimizers/



In [0]:
regressor.summary()

In [0]:
# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 50, batch_size = 32)


# Making the predictions and visualising the results


In [0]:
# Getting the real stock price of 2017
dataset_test = pd.read_csv('Google_Stock_Price_Test.csv')
real_stock_price = dataset_test.iloc[:, 1:2].values


In [0]:
# Getting the predicted stock price of 2017
dataset_total = pd.concat((train_pd['Open'], test_pd['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(test_pd) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []


In [0]:
m = inputs.shape[0]
m

In [0]:
for i in range(60, m):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


In [0]:
predicted_scaled_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_scaled_price)

In [0]:
# Visualising the results
plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()

In [0]:
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

# Visualising the results
plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()